In [2]:
#Load data
from langchain_community.document_loaders import PyPDFDirectoryLoader

doc = PyPDFDirectoryLoader("data/").load()
len(doc)

10

In [4]:
#Chunk data
from langchain_text_splitters import RecursiveCharacterTextSplitter
chunk_data = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap =100).split_documents(doc)
len(chunk_data)

23

In [5]:
#Initialise embeddings
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(api_key=api_key)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001E459E42960>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001E459E42150>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [6]:
#Create VectorStore
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(chunk_data, embeddings)

In [7]:
#Create retriever
query = "What are the key improvements of Llama3 from its predecessors?"

retriever = vector_store.as_retriever()
retriever.get_relevant_documents(query)

c:\Users\arman\OneDrive\Documents\GenAI\RAG-demo\FAISS\myvenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='and deployment of LLMs. We are embracing the open source ethos of releasing early and often to \nenable the community to get access to these models while they are still in development. The text -\nbased models we are releasing today are the first in the Llama 3 collection of models. Our goal in \nthe near future is to make Llama 3 multilingual and multimodal, have longer context, and continue \nto improve overall performance across core LLM capabilities such as reasoning and coding.  \n \n \nState -of-the-art performance  \nOur new 8B and 70B parameter Llama 3 models are a major leap over Llama 2 and establish a new \nstate -of-the-art for LLM models at those scales. Thanks to improvements in pretraining and post -\ntraining, our pretrained and instruction -fine-tuned models are the best models existing today at the \n8B and 70B parameter scale. Improvements in our post -training procedures substantially reduced', metadata={'source': 'data\\Meta Llama 3.pdf', 'p

In [8]:
search_score = vector_store.similarity_search_with_relevance_scores(query, k=3)
search_score

[(Document(page_content='and deployment of LLMs. We are embracing the open source ethos of releasing early and often to \nenable the community to get access to these models while they are still in development. The text -\nbased models we are releasing today are the first in the Llama 3 collection of models. Our goal in \nthe near future is to make Llama 3 multilingual and multimodal, have longer context, and continue \nto improve overall performance across core LLM capabilities such as reasoning and coding.  \n \n \nState -of-the-art performance  \nOur new 8B and 70B parameter Llama 3 models are a major leap over Llama 2 and establish a new \nstate -of-the-art for LLM models at those scales. Thanks to improvements in pretraining and post -\ntraining, our pretrained and instruction -fine-tuned models are the best models existing today at the \n8B and 70B parameter scale. Improvements in our post -training procedures substantially reduced', metadata={'source': 'data\\Meta Llama 3.pdf', '

In [9]:
#Crete DocumentsChains

from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

llm = OpenAI(api_key=api_key, temperature=0)

stuff_chain = RetrievalQA.from_chain_type(llm=llm,
                                          chain_type = "stuff",
                                          retriever = retriever)

stuff_chain.run(query)

c:\Users\arman\OneDrive\Documents\GenAI\RAG-demo\FAISS\myvenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' The key improvements of Llama3 from its predecessors include a major leap in performance, establishment of a new state-of-the-art for LLM models at the 8B and 70B parameter scale, improvements in pretraining and post-training procedures, reduced false refusal rates, improved alignment and increased diversity in model responses, and enhanced capabilities such as reasoning, code generation, and instruction following.'

In [10]:
refine_chain = RetrievalQA.from_chain_type(llm=llm,
                                          chain_type = "refine",
                                          retriever = retriever)

refine_chain.run(query)

'\n\nThe key improvements of Llama3 from its predecessors include: \n1. Embracing the open source ethos and releasing models early and often for community access. This allows for faster innovation and improvement through collaboration with the community. \n2. Making Llama3 multilingual and multimodal, allowing for a wider range of applications and use cases. \n3. Increasing the model size to 8B and 70B parameters, establishing a new state-of-the-art for LLM models at those scales. This allows for more complex and accurate language understanding and generation. \n4. Improvements in pretraining and post-training procedures, resulting in the best models at the 8B and 70B parameter scale. This leads to better performance and results in various tasks. \n5. Substantial reduction in post-training procedures, leading to improved performance in core LLM capabilities such as reasoning and coding. This makes Llama3 more efficient and effective in handling complex language tasks. \n6. Improved per

In [11]:
map_reduce_chain = RetrievalQA.from_chain_type(llm=llm,
                                          chain_type = "map_reduce",
                                          retriever = retriever)

map_reduce_chain.run(query)

' The key improvements of Llama3 from its predecessors include a major leap in performance, reduced false refusal rates, improved alignment and diversity in model responses, and enhanced capabilities such as reasoning, code generation, and instruction following. Additionally, Llama3 will soon be available on various platforms and is expected to introduce new capabilities, longer context windows, additional model sizes, and enhanced performance in the coming months.'

In [12]:
def query_data(query):
    chain1 = stuff_chain.run(query)
    chain2 = refine_chain.run(query)
    chain3 = map_reduce_chain.run(query)

    return chain1, chain2, chain3

In [13]:
import gradio as gr

def clear_textbox():
    return "","","",""

with gr.Blocks(theme=gr.themes.Base(), title= "Chain Types") as demo:
    gr.Markdown(
        """
        # Comparing Different Chain Types
        """)
    
    textbox = gr.Textbox(label="Enter query")

    with gr.Row():
        button_send = gr.Button("Send", variant="Primary")
        button_clear = gr.Button("Clear", variant="Primary")

    with gr.Column():
        output1 = gr.Textbox(lines=1, max_lines=15, label="Response from StuffDocumentChain: ")
        output2 = gr.Textbox(lines=1, max_lines=15, label="Response from RefineDocumentChain: ")
        output3 = gr.Textbox(lines=1, max_lines=15, label="Response from MapReduceDocumentChain: ")

    button_send.click(query_data, textbox, outputs=[output1, output2, output3])
    button_clear.click(clear_textbox, None, outputs=[textbox, output1, output2, output3])

demo.launch()
    

c:\Users\arman\OneDrive\Documents\GenAI\RAG-demo\FAISS\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
